# Capstone project: The Battle of Neighborhoods Week 1

## 1 Introduction
### 1.1 Background
Toronto is the most populous city in Canada and an international centre of business, finance, arts, and culture. Also, it is one of the most multicultural and cosmopolitan cities in the world.

New York is the most populous major city in the United States. It has been described as the cultural, financial, and media capital of the world.

Paris is the capital of France and densely populated. It is also one of major centres of finance, diplomacy, commerce, fashion, science and arts in Europe.

### 1.2 Problem description
These cities share similar characteristics, are multicultural and the most populous and financial capital of their respective countries but how similar they really are? This question can be answer by comparing the neighborhoods and determine how similar or dissimilar these cities are. 
### 1.3 Target audience
People living in New York who are planning to move to a similar city. This comparison will help them to decide if these cities are a good candidate for them to move in.

## 2 Data
First, we will need to know the boroughs and the neighborhoods that exist in each city as well as the latitude and longitude coordinates of each neighborhood.

New York has a total of 5 boroughs and 306 neighborhoods. Toronto officially has 140 neighbourhoods. Paris has 20 boroughs (arrondissements) and 80 neighborhoods (quartiers). These datasets are free online. Here, are the links:

New York:
https://geo.nyu.edu/catalog/nyu_2451_34572

Toronto:
https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M,
http://cocl.us/Geospatial_data

Paris:
https://www.data.gouv.fr/fr/datasets/r/e88c6fda-1d09-42a0-a069-606d3259114e

Next, we will use Foursquare API to explore the neighborhoods and obtain the venues located near by.

### Import library

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

###  2.1 Scrape the website pages and create dataframe

#### Toronto

In [2]:
# Read data
d = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df = d[0]
# Rename Postal code column
df.rename(columns={"Postal code": "PostalCode"}, inplace = True)
# Ignore cells with a borough that is Not assigned
df = df[df.Borough != 'Not assigned']
# Replace ' /' by ','
df = df.assign(Neighborhood=df['Neighborhood'].str.replace(' /', ','))
# Reset index
df.reset_index(drop=True, inplace=True)

In [3]:
## Add Latitude/Longitude
df2 = pd.read_csv('C:/Users/michm/Documents/Jupiter notebook/coursera/Geospatial_Coordinates.csv')
# Rename Postal code column
df2.rename(columns={"Postal Code": "PostalCode"}, inplace = True)


In [4]:
# Merge dataframes
data_toronto = pd.merge(df,df2,on='PostalCode')
# Select only Borough that contain the word 'Toronto'
data_toronto = data_toronto[data_toronto['Borough'].str.contains("Toronto")]
# Reset index
data_toronto.reset_index(drop=True, inplace=True)
data_toronto = data_toronto.drop('PostalCode', 1)
# Add City column
data_toronto['City'] = 'Toronto'
data_toronto.head()

,Borough,Neighborhood,Latitude,Longitude,City
0,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,Toronto
1,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,Toronto
2,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,Toronto
3,Downtown Toronto,St. James Town,43.651494,-79.375418,Toronto
4,East Toronto,The Beaches,43.676357,-79.293031,Toronto


In [5]:
data_toronto.shape

(39, 5)

#### New York

In [6]:
!wget -q -O newyork_data.json https://cocl.us/new_york_dataset 

In [7]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [8]:
neighborhoods_data = newyork_data['features']

In [9]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
data_newyork = pd.DataFrame(columns=column_names)

In [10]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    data_newyork = data_newyork.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [11]:
data_newyork['City'] = 'New York'
data_newyork.head()

,Borough,Neighborhood,Latitude,Longitude,City
0,Bronx,Wakefield,40.894705,-73.847201,New York
1,Bronx,Co-op City,40.874294,-73.829939,New York
2,Bronx,Eastchester,40.887556,-73.827806,New York
3,Bronx,Fieldston,40.895437,-73.905643,New York
4,Bronx,Riverdale,40.890834,-73.912585,New York


In [12]:
data_newyork.shape

(306, 5)

#### Paris

In [73]:
# Load Boroughs in Paris
with open('arrondissements.json') as json_data:
    arrondissements_data = json.load(json_data)

In [74]:
# Load Neighborhoods in Paris
with open('quartier_paris.json') as json_data2:
    quartier_data = json.load(json_data2)

In [75]:
# define the dataframe columns
column_names = ['Borough number', 'Borough name'] 

# instantiate the dataframe
borough_paris = pd.DataFrame(columns=column_names)

In [76]:
for data in arrondissements_data:
    borough_name = data['fields']['l_aroff'] 
    borough_number = data['fields']['c_ar']
               
    borough_paris = borough_paris.append({'Borough number': borough_number,
                                          'Borough name': borough_name}, ignore_index=True)

In [77]:
# define the dataframe columns
column_names = ['Borough number', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods_paris = pd.DataFrame(columns=column_names)

In [78]:
for data in quartier_data:
    neighborhood_name2 = data['fields']['l_qu'] 
    borough_number2 = data['fields']['c_ar']
           
    neighborhood_latlon2 = data['fields']['geom_x_y']
    neighborhood_lat2 = neighborhood_latlon2[0]
    neighborhood_lon2 = neighborhood_latlon2[1]
    
    neighborhoods_paris = neighborhoods_paris.append({'Borough number': borough_number2,
                                          'Neighborhood': neighborhood_name2,
                                          'Latitude': neighborhood_lat2,
                                          'Longitude': neighborhood_lon2}, ignore_index=True)

In [79]:
# Merge dataframes
data_paris = pd.merge(neighborhoods_paris,borough_paris,on='Borough number')
# Select columns
data_paris = data_paris[['Borough name', 'Neighborhood', 'Latitude', 'Longitude']]
# Rename column Borough
data_paris.rename(columns={"Borough name": "Borough"}, inplace = True)
data_paris['City'] = 'Paris'
data_paris.head()

,Borough,Neighborhood,Latitude,Longitude,City
0,Opéra,Rochechouart,48.879812,2.344861,Paris
1,Opéra,Saint-Georges,48.879934,2.332850,Paris
2,Opéra,Chaussée-d'Antin,48.873547,2.332269,Paris
3,Opéra,Faubourg-Montmartre,48.873935,2.343253,Paris
4,Reuilly,Bercy,48.835209,2.386210,Paris


In [80]:
data_paris.shape

(80, 5)

#### 2.2 Merge dataframes of the 3 cities

In [149]:
data_cities = data_toronto.append([data_newyork, data_paris])

In [150]:
data_cities.shape

(425, 5)

#### 2.3 Use geopy library to get the latitude and longitude values of Toronto.

In [23]:
address_toronto = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address_toronto)
latitude_toronto = location.latitude
longitude_toronto = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude_toronto, longitude_toronto))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [24]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude_newyork = location.latitude
longitude_newyork = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude_newyork, longitude_newyork))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


In [25]:
address = 'Paris, France'

geolocator = Nominatim(user_agent="paris_explorer")
location = geolocator.geocode(address)
latitude_paris = location.latitude
longitude_paris = location.longitude
print('The geograpical coordinate of Paris are {}, {}.'.format(latitude_paris, longitude_paris))

The geograpical coordinate of Paris are 48.8566969, 2.3514616.


In [147]:
address = 'Rome, Italy'

geolocator = Nominatim(user_agent="italy_explorer")
location = geolocator.geocode(address)
latitude_rome = location.latitude
longitude_rome = location.longitude
print('The geograpical coordinate of Paris are {}, {}.'.format(latitude_rome, longitude_rome))

The geograpical coordinate of Paris are 41.8933203, 12.4829321.


#### 2.4 Create a maps with neighborhoods superimposed on top.

#### Toronto

In [151]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(data_toronto['Latitude'], data_toronto['Longitude'], data_toronto['Borough'], data_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Paris

In [152]:
# create map of Paris using latitude and longitude values
map_paris = folium.Map(location=[latitude_paris, longitude_paris], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(data_paris['Latitude'], data_paris['Longitude'], data_paris['Borough'], data_paris['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_paris)  
    
map_paris

#### New York

In [154]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude_newyork, longitude_newyork], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(data_newyork['Latitude'], data_newyork['Longitude'], data_newyork['Borough'], data_newyork['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

#### Define Foursquare Credentials and Version

In [27]:
CLIENT_ID = 'LLKS5KEFXOZYPPENEKR3OUSBHRKWBUMM22P0NUVVXPS31YBZ' # your Foursquare ID
CLIENT_SECRET = 'JW0XKD5H43THSOX1JU2MGWPLBKZYYIKFLJLAGIFU2OZHNVYO' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LLKS5KEFXOZYPPENEKR3OUSBHRKWBUMM22P0NUVVXPS31YBZ
CLIENT_SECRET:JW0XKD5H43THSOX1JU2MGWPLBKZYYIKFLJLAGIFU2OZHNVYO


#### 2.5 Explore Neighborhoods in Toronto, New York and Paris


In [28]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT = 100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *toronto_venues*.

In [29]:
cities_venues = getNearbyVenues(names=data_cities['Neighborhood'],
                                   latitudes=data_cities['Latitude'],
                                   longitudes=data_cities['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town,

In [30]:
print(cities_venues.shape)
cities_venues.head()

(16375, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
3,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


#### Let's find out how many unique categories can be curated from all the returned venues

In [31]:
cities_venues[cities_venues['Venue Category']=='Neighborhood'] = 'Neighborhood Restaurant'
print('There are {} uniques categories.'.format(len(cities_venues['Venue Category'].unique())))

There are 491 uniques categories.


#### 2.6 Analyze Each Neighborhood

In [32]:
# one hot encoding
cities_onehot = pd.get_dummies(cities_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
cities_onehot['Neighborhood'] = cities_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [cities_onehot.columns[-1]] + list(cities_onehot.columns[:-1])
cities_onehot = cities_onehot[fixed_columns]

cities_onehot.head()


,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,Alsatian Restaurant,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,Auvergne Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Basque Restaurant,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Breton Restaurant,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burgundian Restaurant,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Canal,Canal Lock,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Carpet Store,Caucasian Restaurant,Cemetery,Ch'ti Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Auditorium,College Basketball Court,College Bookstore,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cooking School,Corsican Restaurant,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Distribution Center,Dive Bar,Dive Spot,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Jiangxi Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Light Rail Station,

And let's examine the new dataframe size.

In [33]:
cities_onehot.shape

(16375, 492)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [34]:
cities_grouped = cities_onehot.groupby('Neighborhood').mean().reset_index()
cities_grouped.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,Alsatian Restaurant,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,Auvergne Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Basque Restaurant,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Breton Restaurant,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burgundian Restaurant,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Canal,Canal Lock,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Carpet Store,Caucasian Restaurant,Cemetery,Ch'ti Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Auditorium,College Basketball Court,College Bookstore,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cooking School,Corsican Restaurant,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Distribution Center,Dive Bar,Dive Spot,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Jiangxi Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Light Rail Station,

#### Let's confirm the new size


In [35]:
cities_grouped.shape

(419, 492)

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [36]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [46]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = cities_grouped['Neighborhood']

for ind in np.arange(cities_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(cities_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allerton,Pizza Place,Department Store,Deli / Bodega,Supermarket,Playground,Mexican Restaurant,Pharmacy,Chinese Restaurant,Discount Store,Grocery Store
1,Amérique,French Restaurant,Supermarket,Bus Stop,Bed & Breakfast,Café,Plaza,Bistro,Pool,Bakery,Park
2,Annadale,American Restaurant,Food,Bagel Shop,Sports Bar,Diner,Restaurant,Pizza Place,Train Station,Zoo Exhibit,English Restaurant
3,Archives,French Restaurant,Hotel,Coffee Shop,Italian Restaurant,Clothing Store,Bar,Art Gallery,Cocktail Bar,Bistro,Tapas Restaurant
4,Arden Heights,Coffee Shop,Bus Stop,Pharmacy,Pizza Place,Deli / Bodega,Zoo Exhibit,Empanada Restaurant,Entertainment Service,English Restaurant,Egyptian Restaurant
